In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.DataFrame({
    "key" : ["A","B",'C','A','B','C'],
    'value1' : range(6),
    'value2' : np.random.randint(0, 100, 6)
})

In [4]:
df

,key,value1,value2
0,A,0,51
1,B,1,86
2,C,2,22
3,A,3,83
4,B,4,63
5,C,5,23


# funcion agrregate

toma todos los agregados indicados sobre estructura de datos, en caso anterior se hizo una sola operacion, todo entre strings y sin su metodo, es decir la funcion directa

In [5]:
df.groupby('key').aggregate(["min", "median", "mean", "max"])

value1                 value2                 
       min median mean max    min median  mean max
key                                               
A        0    1.5  1.5   3     51   67.0  67.0  83
B        1    2.5  2.5   4     63   74.5  74.5  86
C        2    3.5  3.5   5     22   22.5  22.5  23

# aun mas especifico
se puede pasar diccionario indicanto que operacion(es) realizar a cada columna del df

In [6]:
df.groupby('key').aggregate({
    'value1' : ['min','max'],
    'value2' : 'max',
})

value1     value2
       min max    max
key                  
A        0   3     83
B        1   4     86
C        2   5     23

# filter

In [7]:
#calculando std
df.groupby('key').std()

,value1,value2
key,,
A,2.12132,22.627417
B,2.12132,16.263456
C,2.12132,0.707107


In [8]:

def filter_func(x):
    return x['value2'].std()>10

In [9]:
# para realizar operacion hay que alternar entre "" y '' para que todo sea naranaja , no funciona

display('df' , "df.groupby('key').std()" , "df.groupby('key').filter_func()")

'df'

"df.groupby('key').std()"

"df.groupby('key').filter_func()"

In [49]:
df.groupby('key').filter(filter_func)

,key,value1,value2
0,A,0,19
1,B,1,40
3,A,3,64
4,B,4,5


In [11]:
# mismo resultado con labda
df.groupby('key').filter(lambda x: x.value2.std()>10 )


,key,value1,value2
0,A,0,51
1,B,1,86
3,A,3,83
4,B,4,63


Esta funciona filtra df  eliminando las filas pertenecientes al "factor" que no cumplen con la condición dada.

## transform

sirve cuando se quiere version reducida de los datos pero sin combinar, combinar datos de una forma

In [53]:
# funciona con lambda , para todas las variables
df.groupby('key').transform(lambda x: x-x.mean())

,value1,value2
0,-1.5,-22.5
1,-1.5,17.5
2,-1.5,7.0
3,1.5,22.5
4,1.5,-17.5
5,1.5,-7.0


# apply
permite aplicar cualquier funcion a un conjunto de resultados, el parametro tiene que dser df y devuelve pandas, o bien series o escalar

In [14]:
# permite ademas trabajar con una sola columna, no como el anterior

def norm_by_una(x):
    x['value1'] -= x['value1'].mean()
    return x

df.groupby('key').apply(norm_by_una)

/tmp/ipykernel_324894/926749090.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('key').apply(norm_by_una)


key  value1  value2
key                      
A   0   A    -1.5      51
    3   A     1.5      83
B   1   B    -1.5      86
    4   B     1.5      63
C   2   C    -1.5      22
    5   C     1.5      23

In [15]:
# se realiza primero suma de columa value 2 y luego se divide a cada valor
def norm_by_col2(x):
    x['value1'] /= x['value2'].sum()
    return x

In [16]:
# esta funcion si altera datos sacar .copy()
norm_by_col2(df)

,key,value1,value2
0,A,0.000000,51
1,B,0.003049,86
2,C,0.006098,22
3,A,0.009146,83
4,B,0.012195,63
5,C,0.015244,23


In [17]:
df

,key,value1,value2
0,A,0.000000,51
1,B,0.003049,86
2,C,0.006098,22
3,A,0.009146,83
4,B,0.012195,63
5,C,0.015244,23


notemos que en este caso se normalizo cada dato empleando la suma de toda la columna 2, mientras en que el siguiente, se realiza unicamente sumando por los respectivos "factores". permite realizar operaciones entre dos columnas

In [18]:
df.groupby('key').apply(norm_by_col2)

/tmp/ipykernel_324894/828502737.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('key').apply(norm_by_col2)


key    value1  value2
key                        
A   0   A  0.000000      51
    3   A  0.000068      83
B   1   B  0.000020      86
    4   B  0.000082      63
C   2   C  0.000136      22
    5   C  0.000339      23

In [19]:
#cuando se hacen operaciones con groupby no se alteran datos
df


,key,value1,value2
0,A,0.000000,51
1,B,0.003049,86
2,C,0.006098,22
3,A,0.009146,83
4,B,0.012195,63
5,C,0.015244,23


# entendiendo el groupby

en los casos desarrollados, lo que hace es "crear" 3 tablas para sus operaciones, de tal forma que resultan  tablas de la forma

A 

value 1 value 2
   0      70
   3      55

B

value 1 value 2

... 

de forma que las operaciones subsecuentes con apply transform o filter, se aplican a cada tabla

In [97]:
df

,key,value1,value2
0,A,0,70
1,B,1,70
2,C,2,63
3,A,3,55
4,B,4,42
5,C,5,36
